In [1]:
from __future__ import division, print_function, absolute_import
import csv
import pandas
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
import os
import xml.etree.ElementTree as etree
import cobra
import numpy as np
from itertools import chain
from cobra.util import solver as sutil
from cobra.core.solution import get_solution
from optlang.symbolics import add, Zero
import pandas as pd
import os
from os.path import join
import matplotlib.pyplot as plt
from cobra.medium import minimal_medium
# Press ⌃R to execute it or replace it with your code.
# Press Double ⇧ to search everywhere for classes, files, tool windows, actions, and settings.
from cobra.flux_analysis import production_envelope
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import matplotlib.pyplot as plt
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model



In [2]:
## Pareto function
# Pareto
objective1={''}
objective2={''}
pareto_range = (0.0, 1.001)  # for some reason you need to pick a number higher than 1).
pareto_step_size = 0.01
analysis_type = 'pareto'
metric = 'manhattan'
rxn2avoid = {''}
solver='gurobi'
constants = {'deltaC_CO2': 0.0055, 'D_H2O_0': 2.13E-05, 'D_CO2_0': 1.33E-05, 'mid_day': 6, 'deltaT': 2,
             'FeasTol': 1e-03, 'OptTol': 1e-03}
def pareto_analysis(model, objective1=objective1, objective2=objective2, pareto_range=pareto_range, metric=metric):
    reaction_obj1 = model.reactions.get_by_id(objective1)
    reaction_obj2 = model.reactions.get_by_id(objective2)
    result_list = []
    model.objective = {}
    reaction_obj1.objective_coefficient = 1
    solution = model.optimize()
    print("\nSolving model (FBA) for determining objective 1 flux...")
    max_obj1 = dict(solution.fluxes)[objective1]
    print("Max {0}: {1}".format(objective1, max_obj1))
    # change objective
    reaction_obj1.objective_coefficient = 0
    reaction_obj2.objective_coefficient = 1
    print("\nSolving all iterations for Pareto frontier (FBA)...")
    for pareto in np.arange(pareto_range[0], pareto_range[1], pareto_step_size):
        if pareto == 1:
            reaction_obj1.lower_bound = max_obj1 * pareto  # * 0.999 # we need to add a bit of slack as the quadratic optimization is less accurate than the linear couterpart
        else:
            reaction_obj1.lower_bound = max_obj1 * pareto  # * 0.9999
        sol = model.optimize(objective_sense='maximize')
        # fix this minimal water loss value
        reaction_obj2.bounds = (sol.get_primal_by_id(objective2), sol.get_primal_by_id(objective2))
        if metric == 'manhattan':
            solution = cobra.flux_analysis.pfba(model)
            # print({'proline sink': solution['SK_PRO_c_06'], 'biomass 05': solution['Leaf_biomass_tx_05'], 'biomass 06': solution['Leaf_biomass_tx_06']})
            # solution.fluxes.to_excel(f'pareto_no_{pareto}.xlsx')
            result_list.append([pareto, solution[objective1], solution[objective2]])
            reaction_obj2.bounds = (0, 1000.0)
        elif metric == 'euclidean':

            # make copy because that is easier that reverting all the solver settings
            copy_model = model.copy()
            model.solver = solver

            FeasTol = float(constants['FeasTol'])
            OptTol = float(constants['OptTol'])

            copy_model.solver.configuration.tolerances.feasibility = FeasTol
            copy_model.solver.configuration.tolerances.optimality = OptTol

            rxnlist = [r for r in copy_model.reactions if r.id not in rxn2avoid]

            obj_vars = chain.from_iterable([r.flux_expression ** 2] for r in rxnlist)
            copy_model.objective = copy_model.problem.Objective(add(obj_vars), direction='min')

            print('\nSolving quadratic minimisation of sum of fluxes')
            #print(solver)
            solution = copy_model.optimize(objective_sense=None)
            result_list.append([pareto, solution[objective1], solution[objective2]])
        reaction_obj2.bounds = (0, 1000.0)
    return result_list

In [3]:
model_rs = cobra.io.load_matlab_model(join('alpha_day_RS_DM.mat'))
core_model=model_rs


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-16


In [4]:
##Constraints
rubisco = core_model.problem.Constraint(3 * core_model.reactions.get_by_id("RXN_961_p").flux_expression - core_model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p").flux_expression,lb=0, ub=0,)
core_model.add_cons_vars([rubisco])
## 
atp = core_model.problem.Constraint((0.0049*core_model.reactions.get_by_id("Photon_tx").flux_expression+2.7851)-core_model.reactions.get_by_id("ATPase_tx").flux_expression, lb=0, ub=0)
core_model.add_cons_vars(atp)
##
atp_nadph_03 = core_model.problem.Constraint(3 * (core_model.reactions.get_by_id("NADPHoxm_tx").flux_expression + core_model.reactions.get_by_id("NADPHoxc_tx").flux_expression + core_model.reactions.get_by_id("NADPHoxp_tx").flux_expression) - core_model.reactions.get_by_id("ATPase_tx").flux_expression, lb=0, ub=0)
core_model.add_cons_vars(atp_nadph_03)

In [5]:
#Creating an empty dataframe
df_p = []
df_c = []

## H2O2
budget_metabolites=["HYDROGEN_PEROXIDE_c","HYDROGEN_PEROXIDE_m","HYDROGEN_PEROXIDE_p","HYDROGEN_PEROXIDE_x","HYDROGEN_PEROXIDE_v","HYDROGEN_PEROXIDE_e","HYDROGEN_PEROXIDE_g","HYDROGEN_PEROXIDE_n"]
compartment = ["_c", "_m", "_p", "_x","_v","_e","_g","_n"]

#budget_metabolites=["HYDROGEN_PEROXIDE_cell"]
#compartment = ["_cell"]

#budget_metabolites=["SUPER_OXIDE_c","SUPER_OXIDE_m","SUPER_OXIDE_p","SUPER_OXIDE_x","SUPER_OXIDE_v","SUPER_OXIDE_e","SUPER_OXIDE_cell","SUPER_OXIDE_n"]
#compartment = ["_c", "_m", "_p", "_x","_v","_e","_cell","_g"]

c3_model=core_model
for (a,com) in zip(budget_metabolites,compartment):
    i = c3_model.metabolites.get_by_id(a).summary()
    p = i.producing_flux
    c = i.consuming_flux

# Retrieving the reactions using pandas and filtering the transport reactions
    producer = p#.loc[:,"reaction"].str.endswith(com) 
    print(producer)
    df_p.append(producer)
    producer_df = pd.concat(df_p)
        
# Retrieving the reactions using pandas
    consumer = c#.loc[:,"reaction"].str.endswith(com)
    df_c.append(consumer)
    consumer_df = pd.concat(df_c)
consumers=list(consumer_df['reaction'])
producers=list(producer_df.index)

                       flux               reaction  percent
RS_33                   0.0                  RS_33      NaN
RS_34                   0.0                  RS_34      NaN
RS_Plant_ALDO_C         0.0        RS_Plant_ALDO_C      NaN
RS_Plant_PSD2_nad_Mb    0.0   RS_Plant_PSD2_nad_Mb      NaN
RS_Plant_PSD2_nadp_Mb   0.0  RS_Plant_PSD2_nadp_Mb      NaN
SUPEROX_DISMUT_RXN_c    0.0   SUPEROX_DISMUT_RXN_c      NaN
                       flux               reaction  percent
RS_Plant_SOD_M          0.0         RS_Plant_SOD_M      NaN
SULFITE_OXIDASE_RXN_m   0.0  SULFITE_OXIDASE_RXN_m      NaN
                      flux              reaction  percent
RS_35_CP               0.0              RS_35_CP      NaN
RS_46_CP               0.0              RS_46_CP      NaN
RS_74_CP               0.0              RS_74_CP      NaN
SUPEROX_DISMUT_RXN_p   0.0  SUPEROX_DISMUT_RXN_p      NaN
                       flux           reaction  percent
A_B_oxidation_x    0.000000    A_B_oxidation_x      0.

In [6]:
core_model.reactions.get_by_id('DM_HYDROGEN_PEROXIDE_cell').upper_bound=100

In [7]:
p=producer_df['reaction']
p

RS_33                                    RS_33
RS_34                                    RS_34
RS_Plant_ALDO_C                RS_Plant_ALDO_C
RS_Plant_PSD2_nad_Mb      RS_Plant_PSD2_nad_Mb
RS_Plant_PSD2_nadp_Mb    RS_Plant_PSD2_nadp_Mb
SUPEROX_DISMUT_RXN_c      SUPEROX_DISMUT_RXN_c
RS_Plant_SOD_M                  RS_Plant_SOD_M
SULFITE_OXIDASE_RXN_m    SULFITE_OXIDASE_RXN_m
RS_35_CP                              RS_35_CP
RS_46_CP                              RS_46_CP
RS_74_CP                              RS_74_CP
SUPEROX_DISMUT_RXN_p      SUPEROX_DISMUT_RXN_p
A_B_oxidation_x                A_B_oxidation_x
Beta_Oxidation_x              Beta_Oxidation_x
RS_Plant_CAO_X                  RS_Plant_CAO_X
RS_Plant_MSOX_X                RS_Plant_MSOX_X
RS_Plant_PIPOX__X            RS_Plant_PIPOX__X
RS_Plant_SOD_X                  RS_Plant_SOD_X
RS_Plant_URIC_X                RS_Plant_URIC_X
RS_SUOX_X                            RS_SUOX_X
RXN_969_x                            RXN_969_x
RS_Plant_H2O2

In [8]:
coefficients = dict()
for rxn in core_model.reactions[:5]:
    coefficients[rxn.forward_variable] = 1.
    print(coefficients[rxn.forward_variable])
    coefficients[rxn.reverse_variable] = 1.
    print(coefficients[rxn.reverse_variable])

constraint = core_model.problem.Constraint(0, lb=0, ub=0.000001)
core_model.add_cons_vars(constraint)
core_model.solver.update()
constraint.set_linear_coefficients(coefficients=coefficients)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [9]:
met_stoi=[]
for i in producers:
    for met in budget_metabolites:
        try:
            rxn=core_model.reactions.get_by_id(i).get_coefficient(met)
            met_stoi.append(abs(rxn))
        except KeyError:
            continue


In [10]:
met_stoi

[2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 7.0,
 7.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [11]:
## plot pareto plot
objective1 =  'DM_HYDROGEN_PEROXIDE_cell'
objective2 =  'AraCore_Biomass_tx'
solution_primary=pareto_analysis(core_model, objective1 = objective1, objective2=objective2, pareto_range = pareto_range, metric = metric)
#pd.DataFrame(result_list).to_excel('results.xlsx')
data=pd.DataFrame(solution_primary)
print(data)
plt.plot(data[1],data[2]) 
plt.show()
#objs_rs=[AraCore_Biomass_tx 'Phloem_output_tx','DM_NITRIC-OXIDE_cell','DM_HS_cell','DM_SUPER_OXIDE_cell','DM_HC00250_cell','DM_CPD0-1395_cell','DM_SO3_cell','DM_CPD-12377_cell','DM_HYDROGEN_PEROXIDE_cell','DM_ho2_rad_cell']


Solving model (FBA) for determining objective 1 flux...
Max DM_HYDROGEN_PEROXIDE_cell: 100.0

Solving all iterations for Pareto frontier (FBA)...


Infeasible: None (infeasible).